In [28]:
import os # Configure which GPU
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Import Sionna
try:
    import sionna as sn
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna as sn

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

import numpy as np

# For plotting
%matplotlib inline
import matplotlib.pyplot as plt

# For the implementation of the Keras models
from tensorflow.keras import Model

# Set seed for reproducable results
sn.config.seed = 42
import numpy as np

In [87]:
def text_to_bits(text, total_bits=912):
    """Convert a string into a binary bit sequence and pad with zeros to match total_bits."""
    bit_seq = ''.join(format(ord(char), '08b') for char in text)  # Convert to binary (8 bits per char)
    bit_seq = bit_seq.ljust(total_bits, '0')  # Pad with zeros if needed
    return np.array([int(b) for b in bit_seq[:total_bits]]).reshape(1, 1, 1, total_bits)  # Ensure correct shape

def bits_to_text(bits):
    """Convert a binary bit sequence back into a string."""
    bit_seq = ''.join(str(int(b)) for b in bits.flatten())  # Convert back to bit string
    chars = [bit_seq[i:i+8] for i in range(0, len(bit_seq), 8)]  # Split into bytes
    return ''.join(chr(int(char, 2)) for char in chars if int(char, 2) != 0)  # Convert to text, ignore padding

def text_to_bits_tensor(text, total_bits=912):
    """Convert a string into a binary bit sequence, pad to total_bits, and return as a float32 Tensor."""
    bit_seq = ''.join(format(ord(char), '08b') for char in text)  # Convert to binary (8 bits per char)
    bit_seq = bit_seq.ljust(total_bits, '0')  # Pad with zeros if needed

    # Convert to numpy array of shape (1, 1, 1, total_bits) with dtype float32
    bit_array = np.array([int(b) for b in bit_seq[:total_bits]], dtype=np.float32).reshape(1, 1, 1, total_bits)

    # Convert to TensorFlow tensor
    return tf.convert_to_tensor(bit_array, dtype=tf.float32)

    
def bits_tensor_to_text(bit_tensor):
    """Convert a (1,1,1,912) float32 Tensor back to a string."""
    # Convert Tensor to NumPy array
    bit_array = bit_tensor.numpy().astype(int).flatten()  # Convert to 1D integer list
    
    # Convert bits to string
    bit_string = ''.join(str(b) for b in bit_array)

    # Split into 8-bit chunks
    char_bits = [bit_string[i:i+8] for i in range(0, len(bit_string), 8)]
    
    # Convert binary chunks to ASCII characters, ignoring padding zeros
    decoded_text = ''.join(chr(int(char, 2)) for char in char_bits if int(char, 2) != 0)

    return decoded_text

def compute_ber(mbits, bits_hat):
    """Compute Bit Error Rate (BER) by comparing transmitted and received bits."""
    # Convert tensors to NumPy arrays
    mbits_np = mbits.numpy().astype(int)
    bits_hat_np = bits_hat.numpy().astype(int)
    
    # Compute total number of bits
    total_bits = mbits_np.size
    
    # Count bit errors (where transmitted and received bits differ)
    bit_errors = np.sum(mbits_np != bits_hat_np)
    
    # Compute BER
    ber = bit_errors / total_bits
    return ber

In [86]:
def text_to_bits_tensor_16(text, total_bits=1824):
    """Convert a string into a binary bit sequence, pad to total_bits, and return as a float32 Tensor."""
    bit_seq = ''.join(format(ord(char), '08b') for char in text)  # Convert to binary (8 bits per char)
    bit_seq = bit_seq.ljust(total_bits, '0')  # Pad with zeros if needed

    # Convert to numpy array of shape (1, 1, 1, total_bits) with dtype float32
    bit_array = np.array([int(b) for b in bit_seq[:total_bits]], dtype=np.float32).reshape(1, 1, 1, total_bits)

    # Convert to TensorFlow tensor
    return tf.convert_to_tensor(bit_array, dtype=tf.float32)

def bits_tensor_to_text_16(bit_tensor):
    """Convert a (1,1,1,1824) float32 Tensor back to a string."""
    # Convert Tensor to NumPy array
    bit_array = bit_tensor.numpy().astype(int).flatten()  # Convert to 1D integer list
    
    # Convert bits to string
    bit_string = ''.join(str(b) for b in bit_array)

    # Split into 8-bit chunks
    char_bits = [bit_string[i:i+8] for i in range(0, len(bit_string), 8)]
    
    # Convert binary chunks to ASCII characters, ignoring padding zeros
    decoded_text = ''.join(chr(int(char, 2)) for char in char_bits if int(char, 2) != 0)

    return decoded_text

In [88]:
class OFDMSystem(Model): # Inherits from Keras Model

    def __init__(self, perfect_csi):
        super().__init__() # Must call the Keras model initializer

        self.perfect_csi = perfect_csi

        n = int(RESOURCE_GRID.num_data_symbols*NUM_BITS_PER_SYMBOL) # Number of coded bits
        k = int(n*CODERATE) # Number of information bits
        self.k = k

        # The binary source will create batches of information bits
        self.binary_source = sn.utils.BinarySource()

        # The encoder maps information bits to coded bits
        self.encoder = sn.fec.ldpc.LDPC5GEncoder(k, n)

        # The mapper maps blocks of information bits to constellation symbols
        self.mapper = sn.mapping.Mapper("qam", NUM_BITS_PER_SYMBOL)

        # The resource grid mapper maps symbols onto an OFDM resource grid
        self.rg_mapper = sn.ofdm.ResourceGridMapper(RESOURCE_GRID)

        # Frequency domain channel
        self.channel = sn.channel.OFDMChannel(CDL, RESOURCE_GRID, add_awgn=True, normalize_channel=True, return_channel=True)

        # The LS channel estimator will provide channel estimates and error variances
        self.ls_est = sn.ofdm.LSChannelEstimator(RESOURCE_GRID, interpolation_type="nn")

        # The LMMSE equalizer will provide soft symbols together with noise variance estimates
        self.lmmse_equ = sn.ofdm.LMMSEEqualizer(RESOURCE_GRID, STREAM_MANAGEMENT)

        # The demapper produces LLR for all coded bits
        self.demapper = sn.mapping.Demapper("app", "qam", NUM_BITS_PER_SYMBOL)

        # The decoder provides hard-decisions on the information bits
        self.decoder = sn.fec.ldpc.LDPC5GDecoder(self.encoder, hard_out=True)

    @tf.function # Graph execution to speed things up
    def __call__(self, batch_size, ebno_db,bits):
        no = sn.utils.ebnodb2no(ebno_db, num_bits_per_symbol=NUM_BITS_PER_SYMBOL, coderate=CODERATE, resource_grid=RESOURCE_GRID)

        # Transmitter
        #bits = self.binary_source([batch_size, NUM_UT, RESOURCE_GRID.num_streams_per_tx, self.k])
        codewords = self.encoder(bits)
        x = self.mapper(codewords)
        x_rg = self.rg_mapper(x)

        # Channel
        y, h_freq = self.channel([x_rg, no])

        # Receiver
        if self.perfect_csi:
            h_hat, err_var = h_freq, 0.
        else:
            h_hat, err_var = self.ls_est ([y, no])
        x_hat, no_eff = self.lmmse_equ([y, h_hat, err_var, no])
        llr = self.demapper([x_hat, no_eff])
        bits_hat = self.decoder(llr)

        return bits, bits_hat

In [89]:
for val in range(-10,5,1):
    NUM_UT = 1
    NUM_BS = 1
    NUM_UT_ANT = 1
    NUM_BS_ANT = 4
    NUM_STREAMS_PER_TX = NUM_UT_ANT
    RX_TX_ASSOCIATION = np.array([[1]])
    
    STREAM_MANAGEMENT = sn.mimo.StreamManagement(RX_TX_ASSOCIATION, NUM_STREAMS_PER_TX)
    
    NUM_BITS_PER_SYMBOL = 4 # QPSK
    CODERATE = 0.5
    
    RESOURCE_GRID = sn.ofdm.ResourceGrid( num_ofdm_symbols=14,
                                          fft_size=76,
                                          subcarrier_spacing=30e3,
                                          num_tx=NUM_UT,
                                          num_streams_per_tx=NUM_STREAMS_PER_TX,
                                          cyclic_prefix_length=6,
                                          pilot_pattern="kronecker",
                                          pilot_ofdm_symbol_indices=[2,11])
    
    DELAY_SPREAD = 100e-9 
    DIRECTION = "uplink"  
    CDL_MODEL = "C"       # Suitable values are ["A", "B", "C", "D", "E"]
    SPEED = 10.0          # UT speed [m/s]. BSs are always assumed to be fixed.
    
    CARRIER_FREQUENCY = 2.6e9 
    
    UT_ARRAY = sn.channel.tr38901.Antenna(  polarization="single",
                                            polarization_type="V",
                                            antenna_pattern="38.901",
                                            carrier_frequency=CARRIER_FREQUENCY)
    
    BS_ARRAY = sn.channel.tr38901.AntennaArray( num_rows=1,
                                                num_cols=int(NUM_BS_ANT/2),
                                                polarization="dual",
                                                polarization_type="cross",
                                                antenna_pattern="38.901", # Try 'omni'
                                                carrier_frequency=CARRIER_FREQUENCY)
    
    
    CDL = sn.channel.tr38901.CDL(CDL_MODEL,
                                 DELAY_SPREAD,
                                 CARRIER_FREQUENCY,
                                 UT_ARRAY,
                                 BS_ARRAY,
                                 DIRECTION,
                                 min_speed=SPEED)
    
    
    model_ls = OFDMSystem(False)
    mbits, bits_hat = model_ls(1,val,text_to_bits_tensor_16("Athmajan"))
    decoded_word = bits_tensor_to_text(bits_hat)  # Use the Tensor from the previous step
    #print("Decoded Word:", decoded_word)
    print(val)
    print(decoded_word=="Athmajan")
    print(f"BER : {compute_ber(mbits, bits_hat)}")

-10
False
BER : 0.3020833333333333
-9
False
BER : 0.2944078947368421
-8
False
BER : 0.2779605263157895
-7
False
BER : 0.28728070175438597
-6
False
BER : 0.2571271929824561
-5
False
BER : 0.23848684210526316
-4
False
BER : 0.24890350877192982
-3
False
BER : 0.23574561403508773
-2
False
BER : 0.19298245614035087
-1
False
BER : 0.19572368421052633
0
False
BER : 0.11513157894736842
1
False
BER : 0.003289473684210526
2
True
BER : 0.0
3
True
BER : 0.0
4
True
BER : 0.0
